
$$
\huge \text{Konsumenckie załogowe loty kosmiczne „Space-U”}
$$

$$
\Large \text{Temat przewodni: Star Wars – wycieczki z jasną i ciemną stroną mocy}
$$

$$
\large \text{Autorzy: Jagoda Płócienniczak, Magdalena Ryś, Zuzanna Sosnowska, Paweł Stępień, Magdalena Sudół}
$$

$ \textbf{Siedziba firmy: Naboo}$

$ \textbf{Firma na rynku od:} \quad 01.03.2098$

$ \text{Obecny rok:} \quad 01.01.2100$

In [671]:
import mysql.connector

import numpy as np
from collections import defaultdict
import pandas as pd
from datetime import date, timedelta
from typing import List, Tuple, Dict, Optional, Set, Union, Any

In [672]:
current_date = date(2100,1,1)
start_date = date(2098,3,1)

## 1. Podstawowe informacje: adres, rok urodzenia, email, telefon, pensja

- adres : numer (1-200) ulica, dystrykt (założenie: pod jednym adresem może mieszkać max. 5 osób, co odwzorowuje realne warunki mieszkaniowe - rodziny, współlokatorzy itd., unikamy sztucznych skupisk)
- domyślnie określamy, że wszystkie adresy znajdują się na planecie Naboo
- rok urodzenia: wiek z rozkładu normalnego (mu=35, std=10), pasażerowie muszą być w przedziale wiekowym 16-70 (od 16 - dzieci, więc kwestie prawne; 70 - starsi ludzie, więc kwestie zdrowotne)
- email: 3 piewrsze litery imienia, 3 pierwsze litery nazwiska i 4 losowe cyfry, np. Alice White -> aliwhi7290@mail.com
- telefon: 9 losowych cyfr (pierwsza cyfra nie może być 0)
- pensja: podsawowa pensja dla każdego stanowiska + każdy może zarobić losowo do 10% więcej (co oddaje indywidualne premie itd.)

In [673]:
def generate_address_data(
    n: int, max_per_address: int, address_occupancy: Optional[Dict] = None
) -> Tuple[List, Dict]:
    """
    Generuje listę adresów i mapę ich zajętości.

    Args:
        n: Liczba adresów do wygenerowania.
        max_per_address: Maksymalna liczba jednostek na adres.
        address_occupancy: Słownik zawierający informacje o zajętości adresów.

    Returns:
        Krotka zawierająca:
            - listę adresów
            - słownik zajętości adresów
    """
    if address_occupancy is None:
        address_occupancy = defaultdict(int)

    district_street_list = {
        "Theed": ["Amidala", "Organa", "Ruwee", "Binks"],
        "Keren": ["Binks", "Quarsh"],
        "Deeja Peak": ["Panaka", "Thule"],
        "Moenia": ["Typho", "Vos"],
        "Lake Country": ["Naberrie"],
        "Royal District": ["Amidala", "Panaka"],
        "Garden Quarters": ["Organa", "Naberrie"],
    }

    results = []

    while len(results) < n:
        district = np.random.choice(list(district_street_list.keys()))
        street = np.random.choice(district_street_list[district])
        number = np.random.randint(1, 200)

        address = f"{number} {street}"
        key = (address, str(district))

        if address_occupancy[key] < max_per_address:
            results.append(key)
            address_occupancy[key] += 1

    return results, address_occupancy

In [674]:
def generate_birth_date(
    count: int,
    current_year: int = 2100,
    mean_age: int = 35,
    std_dev: int = 10,
    min_age: int = 16,
    max_age: int = 70,
) -> List[date]:
    """
    Generuje listę dat urodzenia na podstawie rozkładu normalnego wieku.

    Args:
        count: Liczba dat do wygenerowania.
        current_year: Aktualny rok.
        mean_age: Średni wiek.
        std_dev: Odchylenie standardowe wieku.
        min_age: Minimalny możliwy wiek.
        max_age: Maksymalny możliwy wiek.

    Returns:
        Lista dat urodzenia.
    """
    age_array = np.random.normal(loc=mean_age, scale=std_dev, size=count).astype(int)
    age_array = np.clip(age_array, min_age, max_age)
    birth_years = current_year - age_array

    months = np.random.randint(1, 13, size=count)
    days_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    days = np.random.randint(1, days_in_month[months - 1] + 1)

    return [date(y, m, d) for y, m, d in zip(birth_years, months, days)]

In [675]:
def generate_email(
    df: pd.DataFrame, existing_emails: Optional[Set[str]] = None
) -> Tuple[List[str], Set[str]]:
    """
    Generuje unikalne adresy email dla jednostek z DataFrame.

    Args:
        df: DataFrame zawierający imiona i nazwiska jednostek.
        existing_emails: Opcjonalny zbiór istniejących adresów email, aby uniknąć duplikatów.

    Returns:
        Krotkę zawierającą:
            - listę wygenerowanych adresów email,
            - zaktualizowany zbiór istniejących adresów email.
    """
    if existing_emails is None:
        existing_emails = set()
    emails = []
    for _, row in df.iterrows():
        base = f"{row['first_name'][:3].lower()}{row['last_name'][:3].lower()}"
        while True:
            digits = "".join(str(np.random.randint(0, 10)) for _ in range(4))
            email = f"{base}{digits}@mail.com"
            if email not in existing_emails:
                existing_emails.add(email)
                emails.append(email)
                break
    return emails, existing_emails

In [676]:
def generate_phone_number(
    count: int, existing_numbers: Optional[Set[str]] = None
) -> Tuple[List[str], Set[str]]:
    """
    Generuje unikalne numery telefonów.

    Args:
        count: Liczba numerów do wygenerowania.
        existing_numbers: Opcjonalny zbiór istniejących numerów, aby uniknąć duplikatów.

    Returns:
        Krotkę zawierającą:
            - listę wygenerowanych numerów,
            - zaktualizowany zbiór istniejących numerów.
    """
    if existing_numbers is None:
        existing_numbers = set()
    numbers = []
    while len(numbers) < count * 2:
        number = str(np.random.randint(1, 10)) + "".join(
            str(d) for d in np.random.randint(0, 10, size=8)
        )
        if number not in existing_numbers:
            existing_numbers.add(number)
            numbers.append(number)
    return numbers, existing_numbers

In [677]:
def generate_salary(df: pd.DataFrame, bonus: float = 0.10) -> pd.Series:
    """
    Generuje wynagrodzenia dla pracowników na podstawie ich stanowisk i bonusu.

    Args:
        df: DataFrame zawierający podstawowe informacje o pracownikach.
        bonus: Maksymalny bonus procentowy od podstawowej pensji.

    Returns:
        Series z obliczonymi wynagrodzeniami.
    """
    base_salaries = {"pilot": 16000, "guide": 8500}
    df = df.copy()
    base_salaries_list = df["position"].map(base_salaries)
    random_bonus_factors = np.random.uniform(0, bonus, size=len(df))
    salary = (base_salaries_list * (1 + random_bonus_factors)).round(2)

    return salary

In [678]:
def generate_personal_data(
    df: pd.DataFrame,
    current_year: int = 2100,
    get_birth_date: bool = True,
    get_email: bool = True,
    get_phone: bool = True,
    get_emergency_phone: bool = True,
    get_salary: bool = False,
    existing_emails: Union[set, None] = None,
    existing_numbers: Union[set, None] = None,
    max_per_address: int = 5,
    address_occupancy: Union[List, None] = None,
) -> Tuple[pd.DataFrame, Union[set, None], Union[set, None], Any]:
    """
    Generuje dane osobowe: data urodzenia, e-mail, numery telefonów, adres i pensja.

    Args:
        df: DataFrame z danymi wejściowymi (lista osób).
        current_year: Obecny rok.
        get_birth_date, get_email, get_phone, get_emergency_phone, get_salary: Flagi określające czy generować konkretne dane.
        existing_emails: Zbiór istniejących e-maili.
        existing_numbers: Zbiór istniejących numerów telefonów.
        max_per_address: Maksymalna liczba mieszkańców na jeden adres.
        address_occupancy: Dotychczasowa lista adresów i ich liczby mieszkańców.

    Returns:
        Tuple zawierający:
            - zmodyfikowany DataFrame z dodanymi kolumnami,
            - zaktualizowany zbiór e-maili,
            - zaktualizowany zbiór numerów telefonów,
            - lista adresów i ich liczby mieszkańców.
    """
    df = df.copy()
    count = len(df)

    # == Datu urodzenia ==
    if get_birth_date:
        df["birth_date"] = generate_birth_date(count, current_year)

    # == Email ==
    if get_email:
        emails, existing_emails = generate_email(df, existing_emails)
        df["email_address"] = emails

    # == Numery telefonów ==
    if get_phone:
        numbers, existing_numbers = generate_phone_number(count, existing_numbers)
        df["phone_number"] = numbers[:count]
        if get_emergency_phone:
            df["emergency_number"] = numbers[count:]

    # == Adresy ==
    address_list, address_occupancy = generate_address_data(
        count, max_per_address, address_occupancy
    )
    addresses, districts = zip(*address_list)
    df["address"] = addresses
    df["district"] = districts

    # == Pensje ==
    if get_salary:
        df["salary"] = generate_salary(df)

    return df, existing_emails, existing_numbers, address_occupancy

## 2. Tabela *employee*

In [679]:
staff_data = pd.DataFrame(
    [
        {
            "first_name": "Han",
            "last_name": "Solo",
            "birth_date": date(2065, 7, 28),
            "position": "pilot",
            "hire_date": date(2098, 3, 11),
            "leave_date": None,
        },
        {
            "first_name": "Jango",
            "last_name": "Fett",
            "birth_date": date(2072, 1, 1),
            "position": "pilot",
            "hire_date": date(2098, 3, 11),
            "leave_date": None,
        },
        {
            "first_name": "Cassian",
            "last_name": "Andor",
            "birth_date": date(2073, 12, 21),
            "position": "pilot",
            "hire_date": date(2099, 5, 22),
            "leave_date": date(2099, 10, 1),
        },
        {
            "first_name": "Ahsoka",
            "last_name": "Tano",
            "birth_date": date(2076, 4, 6),
            "position": "guide",
            "hire_date": date(2098, 3, 1),
            "leave_date": None,
        },
        {
            "first_name": "Sheev",
            "last_name": "Palpatine",
            "birth_date": date(2045, 8, 11),
            "position": "guide",
            "hire_date": date(2098, 4, 15),
            "leave_date": None,
        },
        {
            "first_name": "Ratts",
            "last_name": "Tyerell",
            "birth_date": date(2077, 11, 1),
            "position": "guide",
            "hire_date": date(2098, 3, 1),
            "leave_date": None,
        },
    ]
)

In [680]:
employee_df, existing_emails, existing_numbers, address_occupancy = (
    generate_personal_data(df=staff_data, get_birth_date=False, get_salary=True)
)

## 3. Tabela *address* (employee)

In [681]:
address_list = list(address_occupancy.keys())
address_df = pd.DataFrame(address_list, columns=["address", "district"])

## 4. Tabela *spaceship*

In [682]:
spaceship_df = pd.DataFrame(
    [
        {
            "name": "Aetheris",
            "type": "krążownik",
            "seat_capacity": 8,
            "manufacture_year": 2090,
        },
        {
            "name": "Kleopatra",
            "type": "luksusowy",
            "seat_capacity": 10,
            "manufacture_year": 2097,
        },
        {
            "name": "Falcon-X",
            "type": "transportowiec",
            "seat_capacity": 12,
            "manufacture_year": 2075,
        },
        {
            "name": "Tesserakt",
            "type": "towarowy",
            "seat_capacity": 16,
            "manufacture_year": 2087,
        },
    ]
)

## 5. Tabela *trip_type*

In [683]:
trip_type_df = pd.DataFrame(
    [
        {
            "trip_name": "Escape Room",
            "force_side": "jasna",
            "planet_name": "Gwiazda Śmierci",
            "description": "Opuszczona stacja dryfująca w próżni. Rozwiąż kosmiczne zagadki i odkryj, co się tu wydarzyło.",
            "customer_price_total": 45000,
            "daily_cost_per_participant": 3000,
            "trip_days": 3,
            "distance": 15,
        },
        {
            "trip_name": "Kosmiczne SPA",
            "force_side": "jasna",
            "planet_name": "Scarif",
            "description": "Relaks na tropikalnych plażach, zabiegi regeneracyjne i kąpiele w oceanicznych wodach.",
            "customer_price_total": 60000,
            "daily_cost_per_participant": 3500,
            "trip_days": 10,
            "distance": 9,
        },
        {
            "trip_name": "Głębie oceanów",
            "force_side": "jasna",
            "planet_name": "Kamino",
            "description": "Poznanie fauny i flory morskiego świata oraz zwiedzanie tajemniczego centrum klonowania.",
            "customer_price_total": 40000,
            "daily_cost_per_participant": 2250,
            "trip_days": 4,
            "distance": 11,
        },
        {
            "trip_name": "Złote piaski pustynii",
            "force_side": "jasna",
            "planet_name": "Tatooine",
            "description": "Wyścigi ścigaczy, karawany przez wydmy i noc pod gwiazdami z opowieściami tubylców.",
            "customer_price_total": 35000,
            "daily_cost_per_participant": 2000,
            "trip_days": 7,
            "distance": 6,
        },
        {
            "trip_name": "Survival: Awaria 09",
            "force_side": "ciemna",
            "planet_name": "Kashyyyk",
            "description": "Symulowana awaria w przestrzeni. Ograniczone zasoby, szybkie decyzje - przetrwasz?",
            "customer_price_total": 45000,
            "daily_cost_per_participant": 2500,
            "trip_days": 9,
            "distance": 7,
        },
        {
            "trip_name": "Jaskinie zaginionych",
            "force_side": "ciemna",
            "planet_name": "Geonosis",
            "description": "Naturalne tunele, echa przeszłości i mrok, który żyje własnym życiem. Tylko dla odważnych.",
            "customer_price_total": 25000,
            "daily_cost_per_participant": 2250,
            "trip_days": 3,
            "distance": 5,
        },
        {
            "trip_name": "Śnieżna fantazja",
            "force_side": "ciemna",
            "planet_name": "Hoth",
            "description": "Ekspedycja do opuszczonych baz, jazda na nartach po lodowych zboczach i biwaki w ekstremalnych warunkach.",
            "customer_price_total": 50000,
            "daily_cost_per_participant": 3500,
            "trip_days": 7,
            "distance": 10,
        },
    ]
)

## 6. Tabela *insurance*

In [684]:
insurance_df = pd.DataFrame(
    [
        {
            "type": "Brak",
            "description": "Pakiet dla tych, którzy wierzą, że „moc ich ochroni”. Nie zawiera żadnej ochrony, ale trzymamy kciuki.",
            "price": 0,
        },
        {
            "type": "Standard",
            "description": "Pakiet dla tych, którzy zabierają tylko najpotrzebniejsze rzeczy i uważają, że najgorsze, co ich spotka, to zgubienie dokumentów.",
            "price": 1100,
        },
        {
            "type": "Premium",
            "description": "Pakiet dla tych, którzy wiedzą, że walizka poleci na inną planetę, a prom spóźni się przez burzę w hiperprzestrzeni.",
            "price": 1800,
        },
        {
            "type": "VIP",
            "description": "Pakiet dla tych, którzy zawsze pytają: Czy to na pewno jest bezpieczne?",
            "price": 7000,
        },
    ]
)

## -- WSTAWIANIE DANYCH --

In [685]:
def insert_many(cursor, query, df, columns):
    data = [
        tuple(
            row[col].item() if hasattr(row[col], "item") else row[col]
            for col in columns
        )
        for _, row in df.iterrows()
    ]
    cursor.executemany(query, data)


def fetch_table(cursor, table_name):
    cursor.execute(f"SELECT * FROM `{table_name}`")
    return pd.DataFrame(
        cursor.fetchall(), columns=[col[0] for col in cursor.description]
    )

In [686]:
try:
    con = mysql.connector.connect(
        host="giniewicz.it", user="team12", password="te@mliz", database="team12"
    )
    cursor = con.cursor()

    tables = [
        "accident_report",
        "insurance_coverage",
        "opinion",
        "payment",
        "insurance",
        "spaceflight",
        "trip_type",
        "spaceship",
        "customer",
        "employee",
        "accident_type",
        "address_book",
    ]
    for table in tables:
        cursor.execute(f"DELETE FROM {table}")
        cursor.execute(f"ALTER TABLE {table} AUTO_INCREMENT = 1")

    # == address (employee) ===
    insert_address_query = (
        "INSERT INTO address_book (address, district) VALUES (%s, %s)"
    )
    insert_many(cursor, insert_address_query, address_df, ["address", "district"])

    cursor.execute("SELECT id_address, address, district FROM address_book")
    address_map = {(addr, dist): id_addr for id_addr, addr, dist in cursor.fetchall()}

    # == employee ==
    insert_employee_query = (
        "INSERT INTO employee (first_name, last_name, birth_date, position, salary, phone_number, "
        "email_address, id_address, hire_date, leave_date) "
        "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    )

    employee_data = []
    for _, row in employee_df.iterrows():
        addr_key = (row["address"], row["district"])
        id_address = address_map.get(addr_key)
        employee_data.append(
            (
                row["first_name"],
                row["last_name"],
                row["birth_date"],
                row["position"],
                row["salary"],
                row["phone_number"],
                row["email_address"],
                id_address,
                row["hire_date"],
                row["leave_date"],
            )
        )
    cursor.executemany(insert_employee_query, employee_data)

    # == spaceship ===
    insert_spaceship_query = "INSERT INTO spaceship (name, type, seat_capacity, manufacture_year) VALUES (%s, %s, %s, %s)"
    insert_many(
        cursor,
        insert_spaceship_query,
        spaceship_df,
        ["name", "type", "seat_capacity", "manufacture_year"],
    )

    # == trip_type ===
    insert_trip_type_query = (
        "INSERT INTO trip_type (trip_name, force_side, planet_name, description, customer_price_total, "
        "daily_cost_per_participant, trip_days, distance) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    )
    insert_many(
        cursor,
        insert_trip_type_query,
        trip_type_df,
        [
            "trip_name",
            "force_side",
            "planet_name",
            "description",
            "customer_price_total",
            "daily_cost_per_participant",
            "trip_days",
            "distance",
        ],
    )

    # == insurance ===
    insert_insurance_query = (
        "INSERT INTO insurance (type, description, price) VALUES (%s, %s, %s)"
    )
    insert_many(
        cursor, insert_insurance_query, insurance_df, ["type", "description", "price"]
    )

    # == pobieranie danych ==
    employee_df_sql = fetch_table(cursor, "employee")
    spaceship_df_sql = fetch_table(cursor, "spaceship")
    trip_type_df_sql = fetch_table(cursor, "trip_type")
    insurance_df_sql = fetch_table(cursor, "insurance")

    con.commit()

except mysql.connector.Error as err:
    print(f"Błąd: {err}")
    con.rollback()
finally:
    if cursor:
        cursor.close()
    if con:
        con.close()

## 7. Tabela *spaceflight* 

- wybieramy pracowników do wycieczek/wylotów: pracownik w dniu wylotu jest zatrudniony i wrócił z poprzedniej podróży
- wybieramy statek: statek wrócił z poprzedniej wycieczki, w razie kolejnego wylotu czas po jakim może zostać użyty  ponownie wynosi 3 dni (kwestie techniczne - naprawy, przeglądy itp.)
- koszt organizacji/paliwa: cena paliwa za parsek (tzn. odległośc miejsca wycieczki od Naboo) z rozkładu normalnego (mu=450, std=5)
- obliczamy ilość pasażerów na pokładzie: musi być wykupionych przynajmniej 70% miejsc żeby lot się odbył
- lot może mieć opóźnienia (ale nie może wrócić szybciej)
- zakładamy, że wycieczki w niektórych miesiącach mogą być bardziej popularne niż w innych (co jest odzwierciedlone w przypisanych wagach).

In [687]:
def get_available_employee(
    employee_df_sql: pd.DataFrame,
    spaceflights: List[Dict],
    role: str,
    departure: pd.Timestamp,
    return_date: pd.Timestamp,
    role_key: str,
) -> Optional[pd.Series]:
    """
    Generuje dostępnego pracownika na podaną rolę, biorąc pod uwagę zatrudnienie i kolizję terminów.

    Args:
        employee_df_sql: DataFrame z danymi pracowników.
        spaceflights: Lista istniejących lotów.
        role: Rola pracownika ('pilot', 'guide').
        departure: Data i godzina wylotu.
        return_date: Data i godzina powrotu.
        role_key: Klucz w dict z lotem, ('id_pilot', 'id_guide').

    Returns:
        Wylosowany dostępny pracownik lub None, jeśli nie ma dostępnych.
    """
    # Czy pracownik jest aktualnie zatrudniony
    available_emp_df = employee_df_sql[
        (employee_df_sql["position"] == role)
        & (employee_df_sql["hire_date"] <= departure)
        & (
            (employee_df_sql["leave_date"].isna())
            | (employee_df_sql["leave_date"] > return_date)
        )
    ].copy()

    if available_emp_df.empty:
        return None

    # Czy pracownik nie jest zajęty w tym czasie
    busy_ids = set()
    for flight in spaceflights:
        existing_dep = pd.Timestamp(flight["departure_date"])
        existing_ret = pd.Timestamp(flight["return_date"])

        if not (return_date < existing_dep or departure > existing_ret):
            busy_ids.add(flight[role_key])

    # Usuwamy pracowników, którzy są zajęci
    available_emp_df = available_emp_df[~available_emp_df["id_employee"].isin(busy_ids)]

    if available_emp_df.empty:
        return None

    return available_emp_df.sample().iloc[0]

In [688]:
def get_available_spaceships(
    spaceship_df_sql: pd.DataFrame,
    spaceflights: List[Dict],
    departure: pd.Timestamp,
    return_date: pd.Timestamp,
) -> Optional[pd.Series]:
    """
    Generuje dostępny statek (uwzględniając kolizję terminów).

    Args:
        spaceship_df_sql: DataFrame z danymi statków.
        spaceflights: Lista istniejących lotów.
        departure: Data i godzina wylotu.
        return_date: Data i godzina powrotu.

    Returns:
        Wylosowany dostępny statek lub None.
    """
    available_ship_df = spaceship_df_sql.copy()

    busy_ids = set()
    for flight in spaceflights:
        existing_dep = pd.Timestamp(flight["departure_date"])
        existing_ret = pd.Timestamp(flight["return_date"])

        # statek musi wrócić z innej wycieczki i odczekać min 3 dni (kwestie techniczne)
        if not (
            return_date < existing_dep or departure > existing_ret + timedelta(days=3)
        ):
            busy_ids.add(flight["id_spaceship"])

    available_ship_df = available_ship_df[
        ~available_ship_df["id_spaceship"].isin(busy_ids)
    ]

    if available_ship_df.empty:
        return None

    return available_ship_df.sample().iloc[0]

In [689]:
def biased_random_date(start_date: pd.Timestamp, days_range: int) -> pd.Timestamp:
    """
    Generuje losową datę z określonego zakresu, z różnymi prawdopodobieństwami dla poszczególnych miesięcy.

    Args:
        start_date: Data początkowa, od której zaczyna się zakres.
        days_range: Liczba dni określająca długość zakresu dat.

    Returns:
        Losowa data z zakresu, z uwzględnieniem prawd. dla poszczególnych miesięcy.
    """
    offsets = np.arange(days_range + 1)
    dates = start_date + pd.to_timedelta(offsets, unit='D') 
    months = dates.month

    month_weights = np.array([
        0.02,  # Styczeń
        0.03,  # Luty
        0.05,  # Marzec
        0.05,  # Kwiecień
        0.15,  # Maj
        0.20,  # Czerwiec
        0.20,  # Lipiec
        0.15,  # Sierpień
        0.08,  # Wrzesień
        0.04,  # Październik
        0.02,  # Listopad
        0.01   # Grudzień
    ])

    probs = month_weights[months - 1]
    probs = probs / probs.sum()  

    chosen_offset = np.random.choice(offsets, p=probs)

    return start_date + timedelta(days=int(chosen_offset))

In [690]:
def generate_spaceflight(
    num_flights: int,
    trip_type_df_sql: pd.DataFrame,
    spaceship_df_sql: pd.DataFrame,
    employee_df_sql: pd.DataFrame,
    start_date: pd.Timestamp,
    current_date: pd.Timestamp,
) -> Tuple[pd.DataFrame, int]:
    """
    Generuje listę lotów.

    Args:
        num_flights: Liczba lotów do wygenerowania.
        trip_type_df_sql: DataFrame z typami wycieczek.
        spaceship_df_sql: DataFrame z danymi statków.
        employee_df_sql: DataFrame z pracownikami.
        start_date: Data założenia firmy.
        current_date: Aktualna data

    Returns:
        Krotka zawierająca:
            - DataFrame z wygenerowanymi lotami
            - łączna liczba zajętych miejsc przez pasażerów (na pokładzie statku).
    """
    spaceflights = []
    total_passenger_seats = 0

    start_date = pd.Timestamp(start_date)
    current_date = pd.Timestamp(current_date)

    employee_df_sql["hire_date"] = pd.to_datetime(employee_df_sql["hire_date"])
    employee_df_sql["leave_date"] = pd.to_datetime(employee_df_sql["leave_date"])

    for _ in range(num_flights):
        trip = trip_type_df_sql.sample().iloc[0]
        trip_time = trip["trip_days"]

        # 10% szans na zmianę czasu trwania lotu z powodu sytuacji awaryjnej
        if np.random.rand() < 0.1:
            trip_time += max(0, int(np.floor(np.random.normal(loc=1, scale=2))))

        days_range = (current_date - start_date).days
        # losowe dni/godziny/minuty (z większą popularnością na miesiące maj–sierpień)
        departure_day = biased_random_date(start_date, days_range)
        departure_hour = np.random.randint(0, 24)
        departure_minute = np.random.randint(0, 60)
        departure = pd.Timestamp(departure_day) + pd.Timedelta(
            hours=departure_hour, minutes=departure_minute
        )

        return_day = departure + pd.Timedelta(days=trip_time)
        return_hour = np.random.randint(0, 24)
        return_minute = np.random.randint(0, 60)
        return_date = pd.Timestamp(return_day.normalize()) + pd.Timedelta(
            hours=return_hour, minutes=return_minute
        )

        spaceship = get_available_spaceships(
            spaceship_df_sql, spaceflights, departure, return_date
        )
        if spaceship is None:
            continue

        pilot = get_available_employee(
            employee_df_sql, spaceflights, "pilot", departure, return_date, "id_pilot"
        )
        if pilot is None:
            continue

        guide = get_available_employee(
            employee_df_sql, spaceflights, "guide", departure, return_date, "id_guide"
        )
        if guide is None:
            continue

        seat_capacity = spaceship["seat_capacity"]
        available_seats = seat_capacity - 2

        occupied_seats = np.random.randint(
            int(available_seats * 0.7), available_seats + 1
        )
        total_passenger_seats += occupied_seats

        fuel_cost = np.random.normal(loc=450, scale=5)
        organization_cost = round(fuel_cost * 2 * trip["distance"])

        spaceflights.append(
            {
                "id_trip": trip["id_trip"],
                "id_spaceship": spaceship["id_spaceship"],
                "id_pilot": pilot["id_employee"],
                "id_guide": guide["id_employee"],
                "departure_date": departure.strftime(format="%Y-%m-%d %H:%M"),
                "return_date": return_date.strftime(format="%Y-%m-%d %H:%M"),
                "organization_cost": organization_cost,
                "occupied_seats": occupied_seats,
            }
        )

    return pd.DataFrame(spaceflights), total_passenger_seats

In [691]:
def generate_spaceflight_last_year(
    num_flights: int,
    trip_type_df_sql: pd.DataFrame,
    spaceship_df_sql: pd.DataFrame,
    employee_df_sql: pd.DataFrame,
    start_date: pd.Timestamp,
    current_date: pd.Timestamp,
    min_flights: int = 20,
) -> Tuple[pd.DataFrame, int]:
    """
    Generuje dodatkowe loty jeśli w ostatnim roku nie ma określonego progu.

    Args:
        num_flights: Liczba lotów do wygenerowania w początkowym etapie.
        trip_type_df_sql: DataFrame z typami wycieczek.
        spaceship_df_sql: DataFrame z danymi statków.
        employee_df_sql: DataFrame z pracownikami.
        start_date: Data założenia firmy.
        current_date): Aktualna data.
        min_flights: Minimalna liczba lotów w ostatnim roku. Domyślnie 10.

    Returns:
        Krotka zawierająca:
            - DataFrame z wygenerowanymi lotami
            - łączna liczba zajętych miejsc przez pasażerów (na pokładzie statku).
    """
    start_date = pd.Timestamp(start_date)
    current_date = pd.Timestamp(current_date)
    one_year_ago = current_date - timedelta(days=365)

    spaceflight_df, total_passenger_seats = generate_spaceflight(
        num_flights,
        trip_type_df_sql,
        spaceship_df_sql,
        employee_df_sql,
        start_date,
        current_date,
    )

    spaceflight_df["departure_date_dt"] = pd.to_datetime(
        spaceflight_df["departure_date"]
    )
    last_year_flights_count = len(
        spaceflight_df[spaceflight_df["departure_date_dt"] >= one_year_ago]
    )

    while last_year_flights_count < min_flights:
        additional_flights_df, additional_seats = generate_spaceflight(
            num_flights=1,
            trip_type_df_sql=trip_type_df_sql,
            spaceship_df_sql=spaceship_df_sql,
            employee_df_sql=employee_df_sql,
            start_date=one_year_ago,
            current_date=current_date
        )
        if additional_flights_df.empty:
            break

        spaceflight_df = pd.concat(
            [spaceflight_df, additional_flights_df], ignore_index=True
        )
        total_passenger_seats += additional_seats
        last_year_flights_count += 1

    spaceflight_df.drop(columns=["departure_date_dt"], inplace=True)

    return spaceflight_df, total_passenger_seats

In [692]:
spaceflight_df, total_passenger_seats = generate_spaceflight_last_year(
    22,
    trip_type_df_sql,
    spaceship_df_sql,
    employee_df_sql,
    start_date=start_date,
    current_date=current_date
)

spaceflight_df.head()

,id_trip,id_spaceship,id_pilot,id_guide,departure_date,return_date,organization_cost,occupied_seats
0,4,2,1,6,2098-11-17 14:18,2098-11-24 04:22,5509,7
1,3,2,3,6,2099-06-06 14:07,2099-06-10 20:47,9991,5
2,6,4,2,5,2099-10-11 00:48,2099-10-14 06:57,4459,12
3,2,1,1,4,2099-07-13 18:23,2099-07-23 19:33,8287,5
4,7,2,1,6,2098-06-16 13:11,2098-06-23 01:23,9015,5


In [693]:
spaceflight_df["departure_date_dt"] = pd.to_datetime(spaceflight_df["departure_date"])
spaceflight_df["month"] = spaceflight_df["departure_date_dt"].dt.month

summer_months = [5, 6, 7, 8]
summer_count = spaceflight_df[spaceflight_df["month"].isin(summer_months)].shape[0]
non_summer_count = spaceflight_df[~spaceflight_df["month"].isin(summer_months)].shape[0]

print(f"Liczba lotów w miesiącach letnich (maj–sierpień): {summer_count}")
print(f"Liczba lotów w pozostałych miesiącach: {non_summer_count}")

Liczba lotów w miesiącach letnich (maj–sierpień): 19
Liczba lotów w pozostałych miesiącach: 9


In [694]:
flights_2100 = spaceflight_df[spaceflight_df["departure_date_dt"].dt.year == 2100]

if not flights_2100.empty:
    print(f"Liczba lotów w roku 2100: {flights_2100.shape[0]}")
    print(flights_2100[["departure_date", "return_date"]])
else:
    print("Brak lotów z datą w roku 2100.")

Brak lotów z datą w roku 2100.


## 8. Tabela *customer*

- liczbe pasażerów wyliczamy na podstawie ilości zrealizowanych lotów, tzn. suma zajętych wtedy miejsc na pokładzie
- zakładamy, że losowy procent klientów (np. do 15%) może wrócić na kolejne wycieczki

In [695]:
# Losowy procent powracających klientów
returning_ratio = np.random.uniform(0, 0.2)
num_returning_passengers = int(total_passenger_seats * returning_ratio)

# Liczba klientów, których musimy pozyskać
num_needed_passengers = total_passenger_seats - num_returning_passengers

In [696]:
names_data = pd.read_csv("customer_names.csv")
customer_data = names_data[["First Name", "Last Name"]]
customer_data.columns = ["first_name", "last_name"]

customer_data = customer_data.sample(
    n=num_needed_passengers, random_state=42
).reset_index(drop=True)

customer_df, _, _, address_occupancy2 = generate_personal_data(
    df=customer_data,
    existing_emails=existing_emails,
    existing_numbers=existing_numbers,
    address_occupancy=address_occupancy,
)

## 9. Tabela *address* (customer)

In [697]:
address_list2 = list(address_occupancy2.keys())
address_df_customer = pd.DataFrame(address_list2, columns=["address", "district"])

In [698]:
new_address_df = address_df_customer.merge(
    address_df.drop_duplicates(), how="left", on=["address", "district"], indicator=True
)

new_address_df = new_address_df[new_address_df["_merge"] == "left_only"].drop(
    columns=["_merge"]
)

## -- WSTAWIANIE DANYCH --

In [699]:
try:
    con = mysql.connector.connect(
        host="giniewicz.it", user="team12", password="te@mliz", database="team12"
    )
    cursor = con.cursor()

    # == address (customer) ==
    insert_address_query = (
        "INSERT INTO address_book (address, district) VALUES (%s, %s)"
    )
    insert_many(cursor, insert_address_query, new_address_df, ["address", "district"])

    cursor.execute("SELECT id_address, address, district FROM address_book")
    new_address_map = {
        (addr, dist): id_addr for id_addr, addr, dist in cursor.fetchall()
    }

    # == customer ==
    customer_data = []
    for _, row in customer_df.iterrows():
        addr_key = (row["address"], row["district"])
        id_address = new_address_map.get(addr_key)
        customer_data.append(
            (
                row["first_name"],
                row["last_name"],
                row["birth_date"],
                row["phone_number"],
                row["emergency_number"],
                row["email_address"],
                id_address,
            )
        )

    insert_customer_query = """
        INSERT INTO customer (
            first_name, last_name, birth_date, phone_number, emergency_number,
            email_address, id_address
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    cursor.executemany(insert_customer_query, customer_data)

    # == spaceflight ==
    insert_spaceflight_query = """
        INSERT INTO spaceflight (
            id_trip, id_spaceship, id_pilot, id_guide, departure_date, return_date, organization_cost
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    insert_many(
        cursor,
        insert_spaceflight_query,
        spaceflight_df,
        [
            "id_trip",
            "id_spaceship",
            "id_pilot",
            "id_guide",
            "departure_date",
            "return_date",
            "organization_cost",
        ],
    )

    # == pobieranie danych ==
    customer_df_sql = fetch_table(cursor, "customer")
    spaceflight_df_sql = fetch_table(cursor, "spaceflight")

    con.commit()

except mysql.connector.Error as err:
    print(f"Błąd: {err}")
    if con:
        con.rollback()

finally:
    if cursor:
        cursor.close()
    if con:
        con.close()

## 10. Tabela *payment*
- zakładamy, że jeśli klient zapłacił to poleciał
- każdy klient obowiązkowo ma przypisany lot, jeśli mamy więcej lotów niż klientów losujemy po raz drugi (odniesienie do generowania danych *8. Tabela customer*)
- płatność musi być dokonana w przedziale od 30 do 15 dni przed wylotem

In [700]:
spaceflight_df.insert(0, "id_spaceflight", range(1, len(spaceflight_df) + 1))

spaceflight_df_sql = spaceflight_df_sql.merge(
    spaceflight_df[["id_spaceflight", "occupied_seats"]],
    on="id_spaceflight",
    how="left",
)

In [701]:
def generate_payments(
    spaceflight_df: pd.DataFrame, customer_df: pd.DataFrame, insurance_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Generuje płatności dla klientów na podstawie lotów.

    Args:
        spaceflight_df: DataFrame z lotami.
        customer_df: DataFrame z danymi klientów.
        insurance_df: DataFrame z ubezpieczeniami.

    Returns:
        DataFrame z wygenerowanymi płatnościami.
    """
    payments = []

    no_insurance_id = insurance_df.loc[
        insurance_df["type"] == "Brak", "id_insurance"
    ].values[0]
    insurances_for_sampling = insurance_df[insurance_df["type"] != "Brak"]

    all_customers = customer_df.copy()
    remaining_customers = all_customers.sample(frac=1).reset_index(
        drop=True
    )  # losowa kolejność

    customer_index = 0

    for _, flight in spaceflight_df.iterrows():
        seats_occupied = int(flight["occupied_seats"])
        selected_customers = []

        for _ in range(seats_occupied):
            # pierwszeńswto dla klientów bez lotów
            if customer_index < len(remaining_customers):
                customer = remaining_customers.iloc[customer_index]
                customer_index += 1
            else:
                # jeśli już użyliśmy wszystkich klientów – losujemy z całej puli ponownie
                customer = all_customers.sample(1).iloc[0]

            selected_customers.append(customer)

        for customer in selected_customers:
            prob = np.random.uniform(0, 1)

            # Najbardziej prawdopodobna jest wersja standard a najmniej VIP
            weights = insurances_for_sampling["type"].map(
                {"Standard": 3, "Premium": 2, "VIP": 1}
            )
            insurance = insurances_for_sampling.sample(weights=weights).iloc[0]

            payment_offset = np.random.randint(-30, -15)
            payment_date = pd.to_datetime(flight["departure_date"]) + timedelta(
                days=payment_offset
            )

            insurance_id = (
                insurance["id_insurance"] if prob >= 0.25 else no_insurance_id
            )

            payments.append(
                {
                    "id_spaceflight": flight["id_spaceflight"],
                    "id_customer": customer["id_customer"],
                    "id_insurance": insurance_id,
                    "payment_date": payment_date.strftime("%Y-%m-%d"),
                }
            )

    return pd.DataFrame(payments)

In [702]:
payment_df = generate_payments(spaceflight_df_sql, customer_df_sql, insurance_df_sql)

## 11. Tabela *opinion*
- klient może dodać wiarygodną opinię (taką, którą uznajemy) w ciągu 31 dni po powrocie z wycieczki
- zakładamy, że co drugi klient zostawia opinię
- skala ocen: 1-5

In [703]:
payment_df.insert(0, "id_payment", range(1, len(payment_df) + 1))

payment_df = payment_df.merge(
    spaceflight_df_sql[["id_spaceflight", "return_date"]],
    on="id_spaceflight",
    how="left",
)

In [704]:
def generate_opinion(
    payment_df: pd.DataFrame, current_date: date
) -> pd.DataFrame:
    """
    Generuje opinie na podstawie płatności.

    Args:
        payment_df: DataFrame z płatnościami.
        current_date: Aktualna data. Domyślnie 01-01-2100

    Returns:
        DataFrame z opiniami.
    """
    opinions = []
    n = payment_df.shape[0]
    current_date = pd.Timestamp(current_date)

    # Generujemy opinię dla co drugiej płatności
    for _, payment in payment_df.sample(n // 2).iterrows():

        # Opinie z rozkładu normalnego zaokrąglane do połówek
        rating = np.clip(np.round(np.random.normal(loc=4, scale=1) * 2) / 2, 0.5, 5.0)

        opinion_date = pd.to_datetime(payment["return_date"]) + timedelta(
            days=np.random.randint(31)
        )

        if opinion_date > current_date:
            continue

        opinions.append(
            {
                "id_payment": payment["id_payment"],
                "rating": rating,
                "opinion_date": opinion_date.strftime("%Y-%m-%d"),
            }
        )

    return pd.DataFrame(opinions)

In [705]:
opinion_df = generate_opinion(payment_df, current_date)

## 12. Tabela *accident*

In [706]:
accident_type_df = pd.DataFrame(
    [
        {
            "accident_type": "Wypadek medyczny",
            "description": "Obejmuje urazy fizyczne i nagłe zachorowania powstałe podczas podróży – np. poparzenia mieczem świetlnym, reakcje alergiczne na atmosferę innej planety.",
        },
        {
            "accident_type": "Pomoc w przypadku utraty dokumentów",
            "description": "Dotyczy sytuacji utraty lub kradzieży dokumentów tożsamości – np. zgubiony paszport galaktyczny, przejęcie identyfikatora przez droida.",
        },
        {
            "accident_type": "Zagubiony bagaż",
            "description": "Zalicza się tu utrata bagażu osobistego – np. zostawienie plecaka w kantynie na Tatooine, konfiskata przez celników Imperium.",
        },
        {
            "accident_type": "Opóźnienie lotu",
            "description": "Obejmuje przypadki, gdy występują opóźnienia z przyczyn niezależnych od organizatora – np. awarie hipernapędu, zamknięte porty kosmiczne, blokady handlowe w sektorze.",
        },
        {
            "accident_type": "Klonowanie w razie śmierci",
            "description": "Wycieczka była tak nieziemska, że klient aż odszedł z tego świata. Wersja 2.0 już w inkubatorze",
        },
        {
            "accident_type": "Rabunek kosmiczny",
            "description": "Obejmuje kradzież lub wymuszenie dokonane podczas podróży – np. napad w kantynie, przejęcie sprzętu przez piratów z Kessel, konfiskata mienia przez lokalnych przemytników.",
        },
    ]
)

## -- WSTAWIANIE DANYCH --

In [707]:
try:
    con = mysql.connector.connect(
        host="giniewicz.it", user="team12", password="te@mliz", database="team12"
    )
    cursor = con.cursor()

    # == payment ==
    insert_payment_query = """
        INSERT INTO payment (id_payment, id_spaceflight, id_customer, id_insurance, payment_date)
        VALUES (%s, %s, %s, %s, %s)
    """
    insert_many(
        cursor,
        insert_payment_query,
        payment_df,
        ["id_payment", "id_spaceflight", "id_customer", "id_insurance", "payment_date"],
    )

    # == opinion ==
    insert_opinion_query = """
        INSERT INTO opinion (id_payment, rating, opinion_date)
        VALUES (%s, %s, %s)
    """
    insert_many(
        cursor,
        insert_opinion_query,
        opinion_df,
        ["id_payment", "rating", "opinion_date"],
    )

    # == accident_type ==
    insert_accident_type_query = """
        INSERT INTO accident_type (accident_type, description)
        VALUES (%s, %s)
    """
    insert_many(
        cursor,
        insert_accident_type_query,
        accident_type_df,
        ["accident_type", "description"],
    )

    # == pobieranie danych ==
    payment_df_sql = fetch_table(cursor, "payment")
    accident_type_df_sql = fetch_table(cursor, "accident_type")

    con.commit()

except mysql.connector.Error as err:
    print(f"Błąd: {err}")
    if con:
        con.rollback()

finally:
    if cursor:
        cursor.close()
    if con:
        con.close()

## 13. Tabela *coverage*
- każde ubezpieczenie może pokrywać różne rodzaje wypadków

In [708]:
def generate_insurance_coverage(
    insurance_df: pd.DataFrame, accident_type_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Generuje powiązania (pokrycia) pomiędzy ubezpieczeniami a typami wypadków, które obejmują.

    Args:
        insurance_df: DataFrame z ubezpieczeniami.
        accident_type_df: DataFrame z danymi o typach wypadków.
    Returns:
        DataFrame opisujący, które ubezpieczenia obejmują które wypadki.
    """
    coverage_data = []

    coverage_mapping = {
        "Standard": ["Wypadek medyczny", "Pomoc w przypadku utraty dokumentów"],
        "Premium": [
            "Wypadek medyczny",
            "Pomoc w przypadku utraty dokumentów",
            "Zagubiony bagaż",
            "Opóźnienie lotu",
        ],
        "VIP": [
            "Wypadek medyczny",
            "Pomoc w przypadku utraty dokumentów",
            "Zagubiony bagaż",
            "Opóźnienie lotu",
            "Klonowanie w razie śmierci",
            "Rabunek kosmiczny",
        ],
    }
    for _, insurance in insurance_df.iterrows():
        insurance_type = insurance["type"]
        insurance_id = insurance["id_insurance"]

        covered_types = coverage_mapping.get(insurance_type, [])

        for _, accident in accident_type_df.iterrows():
            accident_type = accident["accident_type"]
            if accident_type in covered_types:
                coverage_data.append(
                    {
                        "id_insurance": insurance_id,
                        "id_accident": accident["id_accident"],
                    }
                )

    return pd.DataFrame(coverage_data)

In [709]:
coverage_df = generate_insurance_coverage(insurance_df_sql, accident_type_df_sql)

## 14. Tabela *accident_report*
- każdy klient jest narażony na różne wypadki - klientowi może przydarzyć się podczas jednej podróży kilka wypadków

In [710]:
def generate_accident_report(
    spaceflight_df: pd.DataFrame,
    insurance_df: pd.DataFrame,
    trip_type_df: pd.DataFrame,
    payment_df: pd.DataFrame,
    accident_type_df: pd.DataFrame,
) -> pd.DataFrame:
    """
    Generuje raport wypadków na podstawie danych lotów, ubezpieczeń, płatności i typów wypadków.

    Args:
        spaceflight_df: DataFrame z danymi lotów.
        insurance_df: DataFrame z danymi ubezpieczeń.
        trip_type_df: DataFrame z typami wycieczek.
        payment_df: DataFrame z płatnościami.
        accident_type_df: DataFrame z typami wypadków.

    Returns:
        DataFrame opisująca które płatności zakończyły się którym wypadkiem.
    """
    accident_map = accident_type_df.set_index("accident_type")["id_accident"].to_dict()

    accident_df = accident_type_df.copy()
    accident_df["prob"] = [0.07, 0.01, 0.04, None, 0.005, 0.002]

    merged_df = (
        payment_df.merge(spaceflight_df, on="id_spaceflight", how="left")
        .merge(trip_type_df[["id_trip", "trip_days"]], on="id_trip", how="left")
        .merge(insurance_df[["id_insurance", "type"]], on="id_insurance", how="left")
    )

    merged_df["departure_date"] = pd.to_datetime(merged_df["departure_date"])
    merged_df["return_date"] = pd.to_datetime(merged_df["return_date"])
    merged_df["planned_return_date"] = merged_df["departure_date"] + pd.to_timedelta(
        merged_df["trip_days"], unit="D"
    )
    merged_df["is_delayed_flight"] = (
        merged_df["return_date"].dt.date > merged_df["planned_return_date"].dt.date
    )

    accident_book_df = pd.DataFrame(columns=["id_payment", "id_accident"])

    for i, row in merged_df.iterrows():
        # Opóźnienie
        if row["is_delayed_flight"]:
            n = len(accident_book_df)
            accident_book_df.loc[n] = {
                "id_payment": row["id_payment"],
                "id_accident": accident_map.get("Opóźnienie lotu"),
            }

        # Inne wypadki
        for _, event in accident_df.iterrows():
            if event["prob"] is not None and np.random.rand() < event["prob"]:
                n = len(accident_book_df)
                accident_id = accident_map.get(event["accident_type"])
                accident_book_df.loc[n] = {
                    "id_payment": row["id_payment"],
                    "id_accident": accident_id,
                }

    return accident_book_df

In [711]:
accident_report_df = generate_accident_report(
    spaceflight_df_sql,
    insurance_df_sql,
    trip_type_df_sql,
    payment_df_sql,
    accident_type_df_sql,
)

## -- WSTAWIANIE DANYCH --

In [712]:
try:
    con = mysql.connector.connect(
        host="giniewicz.it", user="team12", password="te@mliz", database="team12"
    )
    cursor = con.cursor()

    # == insurance_coverage ==
    insert_coverage_query = """
        INSERT INTO insurance_coverage (id_insurance, id_accident)
        VALUES (%s, %s)
    """
    insert_many(
        cursor, insert_coverage_query, coverage_df, ["id_insurance", "id_accident"]
    )

    # == accident_report ==
    insert_acc_report_query = """
        INSERT INTO accident_report (id_payment, id_accident)
        VALUES (%s, %s)
    """
    insert_many(
        cursor,
        insert_acc_report_query,
        accident_report_df,
        ["id_payment", "id_accident"],
    )

    con.commit()

except mysql.connector.Error as err:
    print(f"Błąd: {err}")
    if con:
        con.rollback()

finally:
    if cursor:
        cursor.close()
    if con:
        con.close()